In [1]:
import numpy as np
import xarray as xr

from pyrte_rrtmgp import rrtmgp_gas_optics


ERROR_TOLERANCE = 1e-7

rte_rrtmgp_dir = "../../rrtmgp-data"
clear_sky_example_files = f"{rte_rrtmgp_dir}/examples/rfmip-clear-sky/inputs"

rfmip = xr.load_dataset(
    f"{clear_sky_example_files}/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc"
)
rfmip = rfmip.sel(expt=0)  # only one experiment

kdist = xr.load_dataset(f"{rte_rrtmgp_dir}/rrtmgp-gas-lw-g256.nc")
lw_problem = kdist.gas_optics.load_atmospheric_conditions(rfmip)

lw_problem.sfc_emis = rfmip["surface_emissivity"].data

solver_flux_up, solver_flux_down = lw_problem.rte_solve()

rlu_reference = f"{rte_rrtmgp_dir}/examples/rfmip-clear-sky/reference/rlu_Efx_RTE-RRTMGP-181204_rad-irf_r1i1p1f1_gn.nc"
rld_reference = f"{rte_rrtmgp_dir}/examples/rfmip-clear-sky/reference/rld_Efx_RTE-RRTMGP-181204_rad-irf_r1i1p1f1_gn.nc"

rlu = xr.load_dataset(rlu_reference, decode_cf=False)
ref_flux_up = rlu.isel(expt=0)["rlu"].values

rld = xr.load_dataset(rld_reference, decode_cf=False)
ref_flux_down = rld.isel(expt=0)["rld"].values


assert np.isclose(solver_flux_up, ref_flux_up, atol=ERROR_TOLERANCE).all()
assert np.isclose(solver_flux_down, ref_flux_down, atol=ERROR_TOLERANCE).all()
